In [7]:
import pandas as pd
import numpy as np
from geopy import Nominatim
np.random.seed(42)

In [2]:
#working database
df=pd.read_csv('data/address_mrt_school_cbd_mall.csv')

C:\Users\andre\AppData\Local\Continuum\anaconda3\envs\streamprophet37\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# read unique address
df_unique_address=pd.read_csv('data/unique_address_mrt_school.csv')

In [4]:
import json
import requests

def get_postcode(df,col):
    '''
    This function will obtain the postcode using oneMAP api
    
    Keyword arguments:
    df : df name
    col : name of the column to retrieve
    '''
    locations = list(df[col])
    postcode_list = []
    for location in locations:
        url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+location
        response = requests.get(url)
        data = json.loads(response.text) 
        try:
            postcode_list.append(data['results'][0]['POSTAL'])
        except:
            postcode_list.append(None)
    df["postcode"] = postcode_list
    
    return df

In [ ]:
df_unique_postcode = get_postcode(df_unique_address,'address')

In [ ]:
df_unique_postcode.isnull().sum()

In [ ]:
df_unique_postcode

In [63]:
# we will manually input the values since they are only 111 values 
# and we tried geopy but it's inaccurate
# with respect to initital of 9143

address                     0
latitude                    0
longitude                   0
mrt                         0
nearest_mrt_distance        0
mrt_less_than_half_km       0
mrt_less_than_one_km        0
school                      0
nearest_school_distance     0
school_less_than_one_km     0
postcode                   15
dtype: int64

In [67]:
# write to CSV , manually fill in and re-load
df_unique_postcode.to_csv('data/df_balance_null_postcode.csv',index=False)

In [44]:
df_balance_completed = pd.read_csv('data/df_balance_null_postcode.csv')

## Merge back to df_unique 

In [45]:
df_balance_completed.head()

,address,latitude,longitude,mrt,nearest_mrt_distance,mrt_less_than_half_km,mrt_less_than_one_km,school,nearest_school_distance,school_less_than_one_km,postcode
0,309 ang mo kio ave 1,1.366045,103.836970,bright hill mrt,0.546874,0,2,Catholic High School,1.527978,0,NaN
1,216 ang mo kio ave 1,1.366197,103.841505,mayflower mrt,0.800629,0,2,Catholic High School,1.317890,0,560216
2,211 ang mo kio ave 3,1.369197,103.841667,mayflower mrt,0.620303,0,2,Catholic High School,1.634145,0,560211
3,202 ang mo kio ave 3,1.368446,103.844516,ang mo kio mrt,0.580039,0,2,Catholic High School,1.510899,0,560202
4,235 ang mo kio ave 3,1.366824,103.836491,mayflower mrt,0.513092,0,2,Catholic High School,1.629021,0,560235


In [46]:
#unable to find the balance of 16 
df_balance_completed.isnull().sum()

address                     0
latitude                    0
longitude                   0
mrt                         0
nearest_mrt_distance        0
mrt_less_than_half_km       0
mrt_less_than_one_km        0
school                      0
nearest_school_distance     0
school_less_than_one_km     0
postcode                   16
dtype: int64

In [47]:
# we use address as the primary key to merge postcode over
columns = ['address','postcode']
df_unique=df_balance_completed[columns]

In [48]:
df_unique.head()

,address,postcode
0,309 ang mo kio ave 1,NaN
1,216 ang mo kio ave 1,560216
2,211 ang mo kio ave 3,560211
3,202 ang mo kio ave 3,560202
4,235 ang mo kio ave 3,560235


## Merge unique with original dataset

In [49]:
df = pd.read_csv('data/address_mrt_school_cbd_mall.csv')

C:\Users\andre\AppData\Local\Continuum\anaconda3\envs\streamprophet37\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [50]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,nearest_mrt_distance,mrt_less_than_half_km,mrt_less_than_one_km,school,nearest_school_distance,school_less_than_one_km,CBD_distance,mall,nearest_mall_distance,mall_less_than_one_km
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,improved,1977,9000.0,...,0.546874,0,2,Catholic High School,1.527978,0,9.200287,Jubilee Square,1.130965,0
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,improved,1977,6000.0,...,0.546874,0,2,Catholic High School,1.527978,0,9.200287,Jubilee Square,1.130965,0
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,improved,1977,8000.0,...,0.546874,0,2,Catholic High School,1.527978,0,9.200287,Jubilee Square,1.130965,0
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,improved,1977,6000.0,...,0.546874,0,2,Catholic High School,1.527978,0,9.200287,Jubilee Square,1.130965,0
4,1990-02,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,improved,1977,8000.0,...,0.546874,0,2,Catholic High School,1.527978,0,9.200287,Jubilee Square,1.130965,0


In [51]:
df.columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price',
       'date', 'year_sold', 'month_sold', 'remaining_lease', 'address',
       'latitude', 'longitude', 'mrt', 'nearest_mrt_distance',
       'mrt_less_than_half_km', 'mrt_less_than_one_km', 'school',
       'nearest_school_distance', 'school_less_than_one_km', 'CBD_distance',
       'mall', 'nearest_mall_distance', 'mall_less_than_one_km'],
      dtype='object')

In [52]:
# merge df_unique to df . saves us a lot of time
df = pd.merge(df, df_unique, on='address',how='left')

In [53]:
df.columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price',
       'date', 'year_sold', 'month_sold', 'remaining_lease', 'address',
       'latitude', 'longitude', 'mrt', 'nearest_mrt_distance',
       'mrt_less_than_half_km', 'mrt_less_than_one_km', 'school',
       'nearest_school_distance', 'school_less_than_one_km', 'CBD_distance',
       'mall', 'nearest_mall_distance', 'mall_less_than_one_km', 'postcode'],
      dtype='object')

In [54]:
df.isnull().sum()

month                         0
town                          0
flat_type                     0
block                         0
street_name                   0
storey_range                  0
floor_area_sqm                0
flat_model                    0
lease_commence_date           0
resale_price                  0
date                          0
year_sold                     0
month_sold                    0
remaining_lease               0
address                       0
latitude                      0
longitude                     0
mrt                           0
nearest_mrt_distance          0
mrt_less_than_half_km         0
mrt_less_than_one_km          0
school                        0
nearest_school_distance       0
school_less_than_one_km       0
CBD_distance                  0
mall                          0
nearest_mall_distance         0
mall_less_than_one_km         0
postcode                   1217
dtype: int64

In [43]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,mrt_less_than_half_km,mrt_less_than_one_km,school,nearest_school_distance,school_less_than_one_km,CBD_distance,mall,nearest_mall_distance,mall_less_than_one_km,postcode
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,improved,1977,9000.0,...,0,2,Catholic High School,1.527978,0,9.200287,Jubilee Square,1.130965,0,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,improved,1977,6000.0,...,0,2,Catholic High School,1.527978,0,9.200287,Jubilee Square,1.130965,0,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,improved,1977,8000.0,...,0,2,Catholic High School,1.527978,0,9.200287,Jubilee Square,1.130965,0,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,improved,1977,6000.0,...,0,2,Catholic High School,1.527978,0,9.200287,Jubilee Square,1.130965,0,NaN
4,1990-02,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,improved,1977,8000.0,...,0,2,Catholic High School,1.527978,0,9.200287,Jubilee Square,1.130965,0,NaN


In [62]:
df.sample(10)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,mrt_less_than_half_km,mrt_less_than_one_km,school,nearest_school_distance,school_less_than_one_km,CBD_distance,mall,nearest_mall_distance,mall_less_than_one_km,postcode
64151,1998-10,BUKIT MERAH,5 ROOM,72,TELOK BLANGAH HTS,16 TO 18,121.0,standard,1978,450000.0,...,0,2,Nanyang Primary School,4.987709,0,4.740799,Alexandra Retail Centre,0.948251,1,100072
527266,2017-11,TAMPINES,4 ROOM,102,SIMEI ST 1,04 TO 06,104.0,model a,1988,502888.0,...,1,1,Junyuan Primary School,1.490450,0,12.793923,Eastpoint Mall,0.235153,1,520102
268785,1997-11,KALLANG/WHAMPOA,3 ROOM,3,ST. GEORGE'S RD,07 TO 09,59.0,improved,1976,168000.0,...,1,2,Pei Chun Public School,1.739575,0,4.390602,The Poiz,1.267071,0,321003
293852,2007-03,BISHAN,4 ROOM,313,SHUNFU RD,04 TO 06,95.0,new generation,1985,278000.0,...,1,2,Catholic High School,0.930611,1,7.618038,Thomson Plaza,0.828279,1,570313
411711,1998-12,SERANGOON,EXECUTIVE,319,SERANGOON AVE 2,04 TO 06,146.0,maisonette,1985,463000.0,...,0,2,Pei Chun Public School,2.189779,0,7.785741,NEX,0.471495,1,550319
260182,1995-08,GEYLANG,3 ROOM,121,PAYA LEBAR WAY,04 TO 06,68.0,new generation,1980,104000.0,...,0,3,Eunos Primary School,2.409781,0,5.449670,Paya Lebar Square,1.136699,0,381121
203166,1998-05,CHOA CHU KANG,4 ROOM,6,TECK WHYE AVE,01 TO 03,110.0,model a,1984,240000.0,...,0,1,West Spring Primary School,1.617621,0,15.301769,Teck Whye Shopping Centre,0.228024,2,680006
85186,1993-07,GEYLANG,3 ROOM,58,CIRCUIT RD,07 TO 09,56.0,standard,1969,48000.0,...,2,3,Eunos Primary School,1.995384,0,5.969113,Grantral Mall,0.970226,2,370058
255483,1996-06,CENTRAL AREA,3 ROOM,5,TG PAGAR PLAZA,22 TO 24,59.0,improved,1977,165000.0,...,2,8,Methodist Girls School,2.962064,0,1.280933,100 AM,0.189879,3,81005
446560,1996-01,YISHUN,4 ROOM,716,YISHUN ST 71,07 TO 09,83.0,simplified,1987,200000.0,...,0,1,Riverside Primary School,3.665502,0,15.930480,Northpoint City,0.983980,1,760716


In [60]:
df.to_csv('data/address_mrt_school_cbd_mall_postcode_correct.csv',index=False)

In [63]:
df.sample()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,mrt_less_than_half_km,mrt_less_than_one_km,school,nearest_school_distance,school_less_than_one_km,CBD_distance,mall,nearest_mall_distance,mall_less_than_one_km,postcode
588214,2010-09,BEDOK,5 ROOM,648,JLN TENAGA,04 TO 06,122.0,improved,1993,435000.0,...,0,2,Eunos Primary School,0.885658,1,8.038609,Singapore Post Centre,1.908291,0,410648
